In [ ]:
import random
from datetime import datetime, date, timedelta
from itertools import chain
import pandas as pd
import networkx as nx
from itertools import combinations  
import numpy as np
import time

class CollaborationNetworkGraph:
    def __init__(self, note_df):
        """notes_df is assumed to be sorted by date"""
        self.note_df = note_df
        self.G = nx.Graph()
        self.unique_dates = note_df.date.unique()
        self.__create_graph()
        
    def __create_graph(self):
        for note_date in self.unique_dates:
            notes_for_date = self.note_df.query('date == @note_date')
            discharge_ids_for_date = notes_for_date.discharge_id.unique()
            for discharge_id in discharge_ids_for_date:
                drs_for_discharge_id = notes_for_date.query('discharge_id == @discharge_id').dr.unique()
                for comb in list(combinations(drs_for_discharge_id, 2)): self.__add_edge_to_G(*comb)
                # this is where the calculations may need to go...
                # this method could be public and yield results for each team
        
    def __add_edge_to_G(self, name1, name2):
        edge = (name1, name2)
        data = self.G.get_edge_data(*edge, default=None)
        weight = 1 if data is None else data['weight'] + 1
        self.G.add_edge(*edge, weight=weight)
        
class GraphGenerator:
    def __init__(self, note_df, window_in_days, step_in_days):
        note_df.sort_values('date', inplace=True)
        self.note_df = note_df
        self.DELTA = np.timedelta64(window_in_days, 'D')
        step = np.timedelta64(step_in_days, 'D')
        first_date = note_df['date'].iloc[0]
        last_date = note_df['date'].iloc[-1]
        self.date_range = np.arange(first_date, last_date - self.DELTA, step)
        
    def __iter__(self):
        for start_date in self.date_range:
            end_date = start_date + self.DELTA
            notes_in_window = self.note_df.query('date >= @start_date & date <= @end_date')
            graph = CollaborationNetworkGraph(notes_in_window)
            yield graph

In [ ]:
note_df = pd.read_csv('./data/notes.csv', parse_dates=[2])

WINDOW = 90
STEP = 7

get_graphs = GraphGenerator(note_df, WINDOW, STEP)
    
for graph in get_graphs:
#     clust_coefficients = nx.clustering(graph.G)
#     print(clust_coefficients)